In [9]:
!pip install spacy


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 15.3/15.3 MB 7.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


DEPRECATION: https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.3.0/ru_core_news_sm-3.3.0-py3-none-any.whl#egg=ru_core_news_sm==3.3.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\conda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\conda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\conda\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [11]:
from selenium import webdriver

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup as bs

from time import sleep

from webdriver_manager.chrome import ChromeDriverManager

import csv

import requests

from lxml import html

import re

import unicodedata

import pymorphy2



In [12]:
with open('text_10_words.txt', 'r', encoding='windows-1251') as f:
    f = f.read()

In [13]:
topics = f.split('\n')

topics = [topic.split(': ')[-1] for topic in topics if topic != '']

In [14]:
topics[0]

'федерации медицинской организации помощи образования средств соответствии средства также сфере'

In [15]:
ACCENT_MAPPING = {
    '́': '',
    '̀': '',
    'а́': 'а',
    'а̀': 'а',
    'е́': 'е',
    'ѐ': 'е',
    'и́': 'и',
    'ѝ': 'и',
    'о́': 'о',
    'о̀': 'о',
    'у́': 'у',
    'у̀': 'у',
    'ы́': 'ы',
    'ы̀': 'ы',
    'э́': 'э',
    'э̀': 'э',
    'ю́': 'ю',
    '̀ю': 'ю',
    'я́́': 'я',
    'я̀': 'я',
}
ACCENT_MAPPING = {unicodedata.normalize('NFKC', i): j for i, j in ACCENT_MAPPING.items()}


def unaccentify(s):
    ## чтобы не было проблем с надстрочными символами
    
    source = unicodedata.normalize('NFKC', s)
    for old, new in ACCENT_MAPPING.items():
        source = source.replace(old, new)
    return source

In [16]:
import spacy

C:\conda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\conda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\conda\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [17]:
class googling():
    def __init__(self, spacy_model='ru_core_news_sm'):
        
        self.driver = webdriver.Chrome(ChromeDriverManager().install())  ##  инициализация веб-драйвера
        self.nlp = spacy.load(spacy_model)
        
        
        
    def _parse(self):
        ##  внутренняя функция парсинга
        
        soup = bs(self.driver.page_source, 'html.parser')
        search = soup.find_all('div', class_="yuRUbf")
        for h in search:
            self.names.append(h.get_text())


    
    def find(self, word, pages=3):
        ##  ищет первые 30 (по умолчанию) выдач; кол-во страниц можно настраивать
        
        
        self.word = word
        self.names = [] 
        url = "http://www.google.com/search?q=" + self.word
        self.driver.get(url)
        
        self._parse()  ##  парсим
        
        for n in range(pages-1):
            try:
                ##  поиск дополнительных 20 
                ##  смотрим страницу 2 и 3 в поисковой выдаче
                ##  в будущем нужно выделить в отдельную функцию (если будем смотреть все 30 выдач)

                action = ActionChains(self.driver)
                action.key_down(Keys.END).perform()  ##  скролим страницу вниз


                xpath = '//*[@id="pnnext"]/span[2]'
                element = Wait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                element.click()  ##  нажимаем на "следующий"

                self._parse()  ##  парсим

            except:
                ##  в случае если по этим топикам нет много данных
                pass
        
        self.clean()
        
        
        
    def quit(self):
        ##  функция выхода из веб-драйвера
        ##  по сути дублирует функцию самого драйвера, но в ООП иначе никак
        
        self.driver.quit()
        
        
        
    def _lemmatize(self, text):
        ## лемматизация
    
        doc = self.nlp(unaccentify(text))
        return " ".join([token.lemma_ for token in doc])
        
    
    def clean(self):
        ##  внутренняя функция, призванная очистить от лишних знаков
        ##  надо доработать исходя из того, что требуется в итоге
        
        
        self.result = []

        for name in self.names:
            text = re.split('http', name)[0]
            text = re.sub('[-|a-z|.|,|:|0-9|A-z"]', '', text)
            text = re.sub('  ', ' ', text)
            
            self.result.append(self._lemmatize(text.strip('  ')))
            

    
    def show_topics(self):
        ##  для проверки результатов без сохранения в отдельный файл

        print('QUERY:  ', self.word)
        print('TITLES: \n\n')
        for res in self.result:
            print(res)
        
             
            
    def save(self, kuda):
        ##  сохраняем результаты поисковой выдачи (ОТ ОДНОГО ТОПИКА) в отдельный файл 
        
        source = str(kuda) + '.txt'
        with open(source, 'w', encoding="utf-8") as f:
            f.write(f'QUERY: {self.word}\n')
            f.write('TITLES:\n')
            for res in self.result:
                f.write(res)
                f.write('\n')

In [18]:
yandex_part = googling()



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST chromedriver version for 111.0.5563 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Марк\.wdm\drivers\chromedriver\win32\111.0.5563.64]
<ipython-input-17-5f6160664e6b>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())  ##  инициализация веб-драйвера


In [19]:
!mkdir topics_forcomments

Џ®¤Ї ЇЄ  Ё«Ё д ©« topics_forcomments г¦Ґ бгйҐбвўгҐв.


In [20]:

##  смотрим топик 5


#yandex_part.find(topics[5])
#yandex_part.show_topics()

In [21]:
## смотрим все 10 топиков, записываем результат от каждого топика в отдельный файл
## ищем 40 выдач для каждого топика (просматриваем 4 страницы)

for n, topic in enumerate(topics):
    yandex_part.find(topic, pages=4)
    yandex_part.save(f'topics_forcomments/topic_{n}')

In [22]:
yandex_part.quit()